### Kaggle TensorFlow Speech Competition
Use TensorFlow Simple Audio Recognition tutorial code to create submitssion file.

In [1]:
# download dataset
import os, sys

NOTEBOOK_DIR = '/notebooks'
DATA_DIR = '/notebooks/data'
TRAIN_DIR = DATA_DIR + '/training_example'
TEST_DIR = DATA_DIR + "/test/audio"
KAGGLE_TRAIN_DATA_DIR = DATA_DIR + '/train/audio'

if os.path.isdir(DATA_DIR):
    os.chdir(DATA_DIR)
else:
    os.chdir(NOTEBOOK_DIR)
    os.mkdir('data')
    os.chdir('data')

print (os.getcwd())

# Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# assume that the data is downloaded and extracted

/notebooks/data


In [2]:
# select some files as samples
#os.mkdir('sample/train')
#os.mkdir('sample/valid')
#os.mkdir('sample/result')

### Try speech_command sample

In [3]:
import argparse

import numpy as np
from six.moves import xrange
import tensorflow as tf

import input_data; reload (input_data)
import models
from tensorflow.python.platform import gfile

# Set up argument list
parser = argparse.ArgumentParser()
parser.add_argument('--data_url', type=str,
                    # pylint: disable=line-too-long
                    #default='http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz',
                    default='',
                    # pylint: enable=line-too-long
                    help='Location of speech training data archive on the web.')
parser.add_argument('--data_dir', type=str, default=KAGGLE_TRAIN_DATA_DIR,
                    help="""\
                    Where to download the speech training data to.
                    """)
parser.add_argument('--background_volume', type=float, default=0.1,
                    help="""\
                    How loud the background noise should be, between 0 and 1.
                    """)
parser.add_argument('--background_frequency', type=float, default=0.8,
                    help="""\
                    How many of the training samples have background noise mixed in.
                    """)
parser.add_argument('--silence_percentage', type=float, default=10.0,
                    help="""\
                    How much of the training data should be silence.
                    """)
parser.add_argument('--unknown_percentage', type=float, default=10.0,
                    help="""\
                    How much of the training data should be unknown words.
                    """)
parser.add_argument('--time_shift_ms', type=float, default=100.0,
                    help="""\
                    Range to randomly shift the training audio by in time.
                    """)
parser.add_argument('--testing_percentage', type=int, default=10,
                    help='What percentage of wavs to use as a test set.')
parser.add_argument('--validation_percentage', type=int, default=10,
                    help='What percentage of wavs to use as a validation set.')
parser.add_argument('--sample_rate', type=int, default=16000,
                    help='Expected sample rate of the wavs',)
parser.add_argument('--clip_duration_ms', type=int, default=1000,
                    help='Expected duration in milliseconds of the wavs',)
parser.add_argument('--window_size_ms', type=float, default=30.0,
                    help='How long each spectrogram timeslice is',)
parser.add_argument('--window_stride_ms', type=float, default=10.0,
                    help='How long each spectrogram timeslice is',)
parser.add_argument('--dct_coefficient_count', type=int, default=40,
                    help='How many bins to use for the MFCC fingerprint',)
parser.add_argument('--how_many_training_steps', type=str, default='15000,3000',
                    help='How many training loops to run',)
parser.add_argument('--eval_step_interval', type=int, default=400,
                    help='How often to evaluate the training results.')
parser.add_argument('--learning_rate', type=str, default='0.001,0.0001',
                    help='How large a learning rate to use when training.')
parser.add_argument('--batch_size', type=int, default=100,
                    help='How many items to train with at once',)
parser.add_argument('--summaries_dir', type=str, default=TRAIN_DIR + '/retrain_logs',
                    help='Where to save summary logs for TensorBoard.')
parser.add_argument('--wanted_words', type=str, default='yes,no,up,down,left,right,on,off,stop,go',
                    help='Words to use (others will be added to an unknown label)',)
parser.add_argument('--train_dir', type=str, default=TRAIN_DIR + '/speech_commands_train',
                    help='Directory to write event logs and checkpoint.')
parser.add_argument('--save_step_interval', type=int, default=100,
                    help='Save model checkpoint every save_steps.')
#--start_checkpoint=/tmp/speech_commands_train/conv.ckpt-100 or TRAIN_DIR + '/speech_commands_train/conv.ckpt-18000'
parser.add_argument('--start_checkpoint', type=str, default='',
                    help='If specified, restore this pretrained model before any training.')
parser.add_argument('--model_architecture', type=str, default='conv',
                    help='What model architecture to use')
parser.add_argument('--check_nans', type=bool, default=False,
                    help='Whether to check for invalid numbers during processing')

FLAGS, unparsed = parser.parse_known_args()

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

# Start a new TensorFlow session.
sess = tf.InteractiveSession()

# Begin by making sure we have the training data we need. If you already have
# training data of your own, use `--data_url= ` on the command line to avoid
# downloading.
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
    FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
    FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
    FLAGS.data_url, FLAGS.data_dir, FLAGS.silence_percentage,
    FLAGS.unknown_percentage,
    FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
    FLAGS.testing_percentage, model_settings)
fingerprint_size = model_settings['fingerprint_size']
label_count = model_settings['label_count']
time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)

Extracting to : /notebooks/data/train/audio
Words to work with :
{'sheila': True, 'seven': True, 'right': True, 'house': True, 'up': True, 'down': True, 'zero': True, 'go': True, 'yes': True, 'no': True, 'wow': True, 'six': True, 'two': True, 'bird': True, 'happy': True, 'marvin': True, 'stop': True, 'five': True, 'one': True, 'on': True, 'off': True, 'four': True, 'tree': True, 'dog': True, 'bed': True, 'cat': True, 'nine': True, 'three': True, 'eight': True, 'left': True}
Words in interest :
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']


In [5]:
print (model_settings)
print (audio_processor)
print (fingerprint_size)
print (label_count)
print (time_shift_samples)
print (input_data.prepare_words_list(FLAGS.wanted_words.split(',')))

{'fingerprint_size': 3920, 'dct_coefficient_count': 40, 'sample_rate': 16000, 'window_size_samples': 480, 'spectrogram_length': 98, 'desired_samples': 16000, 'window_stride_samples': 160, 'label_count': 12}
3920
12
1600
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']


In [6]:
# Figure out the learning rates for each training phase. Since it's often
# effective to have high learning rates at the start of training, followed by
# lower levels towards the end, the number of steps and learning rates can be
# specified as comma-separated lists to define the rate at each stage. For
# example --how_many_training_steps=10000,3000 --learning_rate=0.001,0.0001
# will run 13,000 training loops in total, with a rate of 0.001 for the first
# 10,000, and 0.0001 for the final 3,000.
training_steps_list = list(map(int, FLAGS.how_many_training_steps.split(',')))
learning_rates_list = list(map(float, FLAGS.learning_rate.split(',')))
if len(training_steps_list) != len(learning_rates_list):
    raise Exception(
        '--how_many_training_steps and --learning_rate must be equal length '
        'lists, but are %d and %d long instead' % (len(training_steps_list),
                                                   len(learning_rates_list)))

In [7]:
print (training_steps_list)
print (learning_rates_list)
print (FLAGS.model_architecture)

[15000, 3000]
[0.001, 0.0001]
conv


In [8]:
fingerprint_input = tf.placeholder(tf.float32, [None, fingerprint_size], name='fingerprint_input')
logits, dropout_prob = models.create_model(fingerprint_input, model_settings,
                                           FLAGS.model_architecture, is_training=True)
# Define loss and optimizer
ground_truth_input = tf.placeholder(tf.float32, [None, label_count], name='groundtruth_input')

# Optionally we can add runtime checks to spot when NaNs or other symptoms of
# numerical errors start occurring during training.
control_dependencies = []
if FLAGS.check_nans:
    checks = tf.add_check_numerics_ops()
    control_dependencies = [checks]

In [9]:
# Create the back propagation and training evaluation machinery in the graph.
with tf.name_scope('cross_entropy'):
    cross_entropy_mean = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=ground_truth_input, logits=logits))
tf.summary.scalar('cross_entropy', cross_entropy_mean)
    
with tf.name_scope('train'), tf.control_dependencies(control_dependencies):
    learning_rate_input = tf.placeholder(
        tf.float32, [], name='learning_rate_input')
    train_step = tf.train.GradientDescentOptimizer(
        learning_rate_input).minimize(cross_entropy_mean)
    
predicted_indices = tf.argmax(logits, 1)
expected_indices = tf.argmax(ground_truth_input, 1)
correct_prediction = tf.equal(predicted_indices, expected_indices)
confusion_matrix = tf.confusion_matrix(expected_indices, predicted_indices)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', evaluation_step)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [10]:
global_step = tf.contrib.framework.get_or_create_global_step()
increment_global_step = tf.assign(global_step, global_step + 1)

saver = tf.train.Saver(tf.global_variables())

# Merge all the summaries and write them out to /tmp/retrain_logs (by default)
merged_summaries = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                   sess.graph)
validation_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/validation')

Instructions for updating:
Please switch to tf.train.get_or_create_global_step


In [11]:
tf.global_variables_initializer().run()

start_step = 1

if FLAGS.start_checkpoint:
    models.load_variables_from_checkpoint(sess, FLAGS.start_checkpoint)
    start_step = global_step.eval(session=sess)

tf.logging.info('Training from step: %d ', start_step)

# Save graph.pbtxt.
tf.train.write_graph(sess.graph_def, FLAGS.train_dir,
                     FLAGS.model_architecture + '.pbtxt')

# Save list of words.
with gfile.GFile(
    os.path.join(FLAGS.train_dir, FLAGS.model_architecture + '_labels.txt'),
    'w') as f:
    f.write('\n'.join(audio_processor.words_list)+'\n')
    
print ("Word list : ")
print (audio_processor.words_list)

INFO:tensorflow:Training from step: 1 
Word list : 
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']


In [12]:
# Training loop.
training_steps_max = np.sum(training_steps_list)
for training_step in xrange(start_step, training_steps_max + 1):
    # Figure out what the current learning rate is.
    training_steps_sum = 0
    for i in range(len(training_steps_list)):
        training_steps_sum += training_steps_list[i]
        if training_step <= training_steps_sum:
            learning_rate_value = learning_rates_list[i]
            break
    # Pull the audio samples we'll use for training.
    train_fingerprints, train_ground_truth = audio_processor.get_data(
        FLAGS.batch_size, 0, model_settings, FLAGS.background_frequency,
        FLAGS.background_volume, time_shift_samples, 'training', sess)
    # Run the graph with this batch of training data.
    train_summary, train_accuracy, cross_entropy_value, _, _ = sess.run(
        [
            merged_summaries, evaluation_step, cross_entropy_mean, train_step,
            increment_global_step
        ],
        feed_dict={
            fingerprint_input: train_fingerprints,
            ground_truth_input: train_ground_truth,
            learning_rate_input: learning_rate_value,
            dropout_prob: 0.5
        })
    train_writer.add_summary(train_summary, training_step)
    if (training_step % 10 == 0):
        tf.logging.info('Step #%d: rate %f, accuracy %.1f%%, cross entropy %f' %
                        (training_step, learning_rate_value, train_accuracy * 100,
                         cross_entropy_value))
    is_last_step = (training_step == training_steps_max)
    if (training_step % FLAGS.eval_step_interval) == 0 or is_last_step:
      set_size = audio_processor.set_size('validation')
      total_accuracy = 0
      total_conf_matrix = None
      for i in xrange(0, set_size, FLAGS.batch_size):
        validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(FLAGS.batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
        # Run a validation step and capture training summaries for TensorBoard
        # with the `merged` op.
        validation_summary, validation_accuracy, conf_matrix = sess.run(
            [merged_summaries, evaluation_step, confusion_matrix],
            feed_dict={
                fingerprint_input: validation_fingerprints,
                ground_truth_input: validation_ground_truth,
                dropout_prob: 1.0
            })
        validation_writer.add_summary(validation_summary, training_step)
        batch_size = min(FLAGS.batch_size, set_size - i)
        total_accuracy += (validation_accuracy * batch_size) / set_size
        if total_conf_matrix is None:
          total_conf_matrix = conf_matrix
        else:
          total_conf_matrix += conf_matrix
      tf.logging.info('Confusion Matrix:\n %s' % (total_conf_matrix))
      tf.logging.info('Step %d: Validation accuracy = %.1f%% (N=%d)' %
                      (training_step, total_accuracy * 100, set_size))

    # Save the model checkpoint periodically.
    if (training_step % FLAGS.save_step_interval == 0 or
        training_step == training_steps_max):
      checkpoint_path = os.path.join(FLAGS.train_dir,
                                     FLAGS.model_architecture + '.ckpt')
      tf.logging.info('Saving to "%s-%d"', checkpoint_path, training_step)
      saver.save(sess, checkpoint_path, global_step=training_step)


INFO:tensorflow:Step #10: rate 0.001000, accuracy 12.0%, cross entropy 2.463859
INFO:tensorflow:Step #20: rate 0.001000, accuracy 14.0%, cross entropy 2.468007
INFO:tensorflow:Step #30: rate 0.001000, accuracy 18.0%, cross entropy 2.379226
INFO:tensorflow:Step #40: rate 0.001000, accuracy 11.0%, cross entropy 2.481290
INFO:tensorflow:Step #50: rate 0.001000, accuracy 15.0%, cross entropy 2.393271
INFO:tensorflow:Step #60: rate 0.001000, accuracy 12.0%, cross entropy 2.423031
INFO:tensorflow:Step #70: rate 0.001000, accuracy 8.0%, cross entropy 2.448517
INFO:tensorflow:Step #80: rate 0.001000, accuracy 16.0%, cross entropy 2.390238
INFO:tensorflow:Step #90: rate 0.001000, accuracy 19.0%, cross entropy 2.417106
INFO:tensorflow:Step #100: rate 0.001000, accuracy 16.0%, cross entropy 2.415254
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-100"
INFO:tensorflow:Step #110: rate 0.001000, accuracy 17.0%, cross entropy 2.376145
INFO:tensorflow:Step #

INFO:tensorflow:Step 800: Validation accuracy = 43.2% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-800"
INFO:tensorflow:Step #810: rate 0.001000, accuracy 29.0%, cross entropy 2.116745
INFO:tensorflow:Step #820: rate 0.001000, accuracy 42.0%, cross entropy 1.903266
INFO:tensorflow:Step #830: rate 0.001000, accuracy 23.0%, cross entropy 2.063649
INFO:tensorflow:Step #840: rate 0.001000, accuracy 31.0%, cross entropy 2.051799
INFO:tensorflow:Step #850: rate 0.001000, accuracy 38.0%, cross entropy 1.847333
INFO:tensorflow:Step #860: rate 0.001000, accuracy 35.0%, cross entropy 1.940476
INFO:tensorflow:Step #870: rate 0.001000, accuracy 36.0%, cross entropy 1.969298
INFO:tensorflow:Step #880: rate 0.001000, accuracy 38.0%, cross entropy 1.940545
INFO:tensorflow:Step #890: rate 0.001000, accuracy 39.0%, cross entropy 1.844210
INFO:tensorflow:Step #900: rate 0.001000, accuracy 34.0%, cross entropy 1.925472
INFO:tensorflow:Saving to "/no

INFO:tensorflow:Step 1600: Validation accuracy = 58.5% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-1600"
INFO:tensorflow:Step #1610: rate 0.001000, accuracy 53.0%, cross entropy 1.612334
INFO:tensorflow:Step #1620: rate 0.001000, accuracy 47.0%, cross entropy 1.592974
INFO:tensorflow:Step #1630: rate 0.001000, accuracy 47.0%, cross entropy 1.533326
INFO:tensorflow:Step #1640: rate 0.001000, accuracy 50.0%, cross entropy 1.503675
INFO:tensorflow:Step #1650: rate 0.001000, accuracy 53.0%, cross entropy 1.455992
INFO:tensorflow:Step #1660: rate 0.001000, accuracy 44.0%, cross entropy 1.473393
INFO:tensorflow:Step #1670: rate 0.001000, accuracy 49.0%, cross entropy 1.477280
INFO:tensorflow:Step #1680: rate 0.001000, accuracy 49.0%, cross entropy 1.504184
INFO:tensorflow:Step #1690: rate 0.001000, accuracy 56.0%, cross entropy 1.364801
INFO:tensorflow:Step #1700: rate 0.001000, accuracy 51.0%, cross entropy 1.618192
INFO:tensorflow:Sa

INFO:tensorflow:Step 2400: Validation accuracy = 69.7% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-2400"
INFO:tensorflow:Step #2410: rate 0.001000, accuracy 59.0%, cross entropy 1.176278
INFO:tensorflow:Step #2420: rate 0.001000, accuracy 56.0%, cross entropy 1.212356
INFO:tensorflow:Step #2430: rate 0.001000, accuracy 61.0%, cross entropy 1.160365
INFO:tensorflow:Step #2440: rate 0.001000, accuracy 66.0%, cross entropy 1.172603
INFO:tensorflow:Step #2450: rate 0.001000, accuracy 62.0%, cross entropy 1.294099
INFO:tensorflow:Step #2460: rate 0.001000, accuracy 56.0%, cross entropy 1.195302
INFO:tensorflow:Step #2470: rate 0.001000, accuracy 66.0%, cross entropy 1.159188
INFO:tensorflow:Step #2480: rate 0.001000, accuracy 59.0%, cross entropy 1.296628
INFO:tensorflow:Step #2490: rate 0.001000, accuracy 60.0%, cross entropy 1.230209
INFO:tensorflow:Step #2500: rate 0.001000, accuracy 60.0%, cross entropy 1.205053
INFO:tensorflow:Sa

INFO:tensorflow:Step 3200: Validation accuracy = 74.5% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-3200"
INFO:tensorflow:Step #3210: rate 0.001000, accuracy 59.0%, cross entropy 1.199411
INFO:tensorflow:Step #3220: rate 0.001000, accuracy 70.0%, cross entropy 0.925828
INFO:tensorflow:Step #3230: rate 0.001000, accuracy 68.0%, cross entropy 1.004959
INFO:tensorflow:Step #3240: rate 0.001000, accuracy 69.0%, cross entropy 1.085253
INFO:tensorflow:Step #3250: rate 0.001000, accuracy 63.0%, cross entropy 1.054895
INFO:tensorflow:Step #3260: rate 0.001000, accuracy 68.0%, cross entropy 1.048456
INFO:tensorflow:Step #3270: rate 0.001000, accuracy 64.0%, cross entropy 1.142398
INFO:tensorflow:Step #3280: rate 0.001000, accuracy 67.0%, cross entropy 0.916596
INFO:tensorflow:Step #3290: rate 0.001000, accuracy 66.0%, cross entropy 1.004719
INFO:tensorflow:Step #3300: rate 0.001000, accuracy 70.0%, cross entropy 1.048464
INFO:tensorflow:Sa

...

INFO:tensorflow:Step #14870: rate 0.001000, accuracy 84.0%, cross entropy 0.483163
INFO:tensorflow:Step #14880: rate 0.001000, accuracy 90.0%, cross entropy 0.396847
INFO:tensorflow:Step #14890: rate 0.001000, accuracy 85.0%, cross entropy 0.570858
INFO:tensorflow:Step #14900: rate 0.001000, accuracy 87.0%, cross entropy 0.371893
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-14900"
INFO:tensorflow:Step #14910: rate 0.001000, accuracy 90.0%, cross entropy 0.374768
INFO:tensorflow:Step #14920: rate 0.001000, accuracy 85.0%, cross entropy 0.493186
INFO:tensorflow:Step #14930: rate 0.001000, accuracy 83.0%, cross entropy 0.526333
INFO:tensorflow:Step #14940: rate 0.001000, accuracy 82.0%, cross entropy 0.640363
INFO:tensorflow:Step #14950: rate 0.001000, accuracy 88.0%, cross entropy 0.363757
INFO:tensorflow:Step #14960: rate 0.001000, accuracy 88.0%, cross entropy 0.372379
INFO:tensorflow:Step #14970: rate 0.001000, accuracy 82.0%, cross entro

INFO:tensorflow:Step 15600: Validation accuracy = 88.5% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-15600"
INFO:tensorflow:Step #15610: rate 0.000100, accuracy 85.0%, cross entropy 0.396374
INFO:tensorflow:Step #15620: rate 0.000100, accuracy 85.0%, cross entropy 0.431713
INFO:tensorflow:Step #15630: rate 0.000100, accuracy 87.0%, cross entropy 0.483854
INFO:tensorflow:Step #15640: rate 0.000100, accuracy 85.0%, cross entropy 0.419257
INFO:tensorflow:Step #15650: rate 0.000100, accuracy 77.0%, cross entropy 0.629599
INFO:tensorflow:Step #15660: rate 0.000100, accuracy 85.0%, cross entropy 0.549874
INFO:tensorflow:Step #15670: rate 0.000100, accuracy 88.0%, cross entropy 0.474887
INFO:tensorflow:Step #15680: rate 0.000100, accuracy 91.0%, cross entropy 0.342047
INFO:tensorflow:Step #15690: rate 0.000100, accuracy 79.0%, cross entropy 0.530155
INFO:tensorflow:Step #15700: rate 0.000100, accuracy 80.0%, cross entropy 0.675705
INFO:t

INFO:tensorflow:Confusion Matrix:
 [[258   0   0   0   0   0   0   0   0   0   0   0]
 [  1 187   4   5   3  10   5  11  16   3   4   9]
 [  3   4 245   5   0   1   2   0   0   0   0   1]
 [  0   7   3 238   2   2   0   0   0   0   2  16]
 [  4   5   0   0 236   0   1   2   1   4   7   0]
 [  0   6   3  19   0 227   1   0   0   0   2   6]
 [  1   2  15   2   0   0 221   5   0   0   1   0]
 [  1  11   0   1   0   0   5 238   0   0   0   0]
 [  4   3   0   0   3   2   0   2 240   2   0   1]
 [  0   3   0   0  20   0   2   3   3 221   3   1]
 [  2   4   0   4   8   1   5   0   0   2 217   3]
 [  6   8   0  20   1   7   2   4   1   2   0 209]]
INFO:tensorflow:Step 16400: Validation accuracy = 88.5% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-16400"
INFO:tensorflow:Step #16410: rate 0.000100, accuracy 88.0%, cross entropy 0.531910
INFO:tensorflow:Step #16420: rate 0.000100, accuracy 88.0%, cross entropy 0.470952
INFO:tensorflow:Step #

INFO:tensorflow:Step #17130: rate 0.000100, accuracy 89.0%, cross entropy 0.462097
INFO:tensorflow:Step #17140: rate 0.000100, accuracy 90.0%, cross entropy 0.413087
INFO:tensorflow:Step #17150: rate 0.000100, accuracy 81.0%, cross entropy 0.666166
INFO:tensorflow:Step #17160: rate 0.000100, accuracy 84.0%, cross entropy 0.464779
INFO:tensorflow:Step #17170: rate 0.000100, accuracy 89.0%, cross entropy 0.320491
INFO:tensorflow:Step #17180: rate 0.000100, accuracy 91.0%, cross entropy 0.319161
INFO:tensorflow:Step #17190: rate 0.000100, accuracy 86.0%, cross entropy 0.361596
INFO:tensorflow:Step #17200: rate 0.000100, accuracy 85.0%, cross entropy 0.473425
INFO:tensorflow:Confusion Matrix:
 [[258   0   0   0   0   0   0   0   0   0   0   0]
 [  1 187   5   6   3  10   5   9  16   3   4   9]
 [  3   2 246   5   0   1   3   0   0   0   0   1]
 [  0   6   3 238   2   2   0   0   0   0   3  16]
 [  4   5   0   0 239   0   1   2   0   3   6   0]
 [  0   5   3  20   0 225   1   0   0   0   4 

INFO:tensorflow:Step #17870: rate 0.000100, accuracy 85.0%, cross entropy 0.432054
INFO:tensorflow:Step #17880: rate 0.000100, accuracy 87.0%, cross entropy 0.495767
INFO:tensorflow:Step #17890: rate 0.000100, accuracy 86.0%, cross entropy 0.338354
INFO:tensorflow:Step #17900: rate 0.000100, accuracy 82.0%, cross entropy 0.623183
INFO:tensorflow:Saving to "/notebooks/data/training_example/speech_commands_train/conv.ckpt-17900"
INFO:tensorflow:Step #17910: rate 0.000100, accuracy 80.0%, cross entropy 0.555462
INFO:tensorflow:Step #17920: rate 0.000100, accuracy 90.0%, cross entropy 0.370045
INFO:tensorflow:Step #17930: rate 0.000100, accuracy 88.0%, cross entropy 0.418754
INFO:tensorflow:Step #17940: rate 0.000100, accuracy 82.0%, cross entropy 0.530795
INFO:tensorflow:Step #17950: rate 0.000100, accuracy 89.0%, cross entropy 0.373481
INFO:tensorflow:Step #17960: rate 0.000100, accuracy 87.0%, cross entropy 0.426397
INFO:tensorflow:Step #17970: rate 0.000100, accuracy 83.0%, cross entro

In [13]:
set_size = audio_processor.set_size('testing')
tf.logging.info('set_size=%d', set_size)
total_accuracy = 0
total_conf_matrix = None
for i in xrange(0, set_size, FLAGS.batch_size):
    test_fingerprints, test_ground_truth = audio_processor.get_data(
        FLAGS.batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
    test_accuracy, conf_matrix = sess.run(
        [evaluation_step, confusion_matrix],
        feed_dict={
            fingerprint_input: test_fingerprints,
            ground_truth_input: test_ground_truth,
            dropout_prob: 1.0
        })
    batch_size = min(FLAGS.batch_size, set_size - i)
    total_accuracy += (test_accuracy * batch_size) / set_size
    if total_conf_matrix is None:
        total_conf_matrix = conf_matrix
    else:
        total_conf_matrix += conf_matrix
tf.logging.info('Confusion Matrix:\n %s' % (total_conf_matrix))
tf.logging.info('Final test accuracy = %.1f%% (N=%d)' % (total_accuracy * 100,
                                                         set_size))

INFO:tensorflow:set_size=3081
INFO:tensorflow:Confusion Matrix:
 [[257   0   0   0   0   0   0   0   0   0   0   0]
 [  2 189   2   1   6   9  10   9   8   0   6  15]
 [  0   4 234   2   1   0  11   2   0   1   1   0]
 [  1   3   3 213   2   7   3   3   0   0   0  17]
 [  0   2   2   0 253   1   2   0   0   3   7   2]
 [  2   4   1   9   1 224   2   0   2   0   0   8]
 [  0   1  16   0   3   1 244   1   0   0   1   0]
 [  1   8   0   0   2   3   4 239   0   1   1   0]
 [  0   4   0   0   1   1   1   1 233   3   1   1]
 [  1   3   0   0   9   0   2   1   7 233   5   1]
 [  0   5   1   0   9   4   2   0   0   1 225   2]
 [  0   8   1  25   4   6   5   2   0   0   2 198]]
INFO:tensorflow:Final test accuracy = 89.0% (N=3081)


### Use the trained model to make a prediction

In [14]:
import csv

# use the model to predict
os.chdir (TEST_DIR)
print (os.getcwd())

file_names = os.listdir(TEST_DIR)
#file_names = map(lambda x: TEST_DIR + '/' + x, os.listdir(TEST_DIR))
#print (len(file_names))
#print (file_names[:10])
words_list = input_data.prepare_words_list(FLAGS.wanted_words.split(','))
#print (words_list)
set_size = len(file_names)
subm = zip(file_names, ('silence' for x in range(set_size)))
#print (len(subm))
#print (subm[:5])

# process the audio files
batch_num = 0
for i in xrange(0, set_size, FLAGS.batch_size):
    batch_size = min(FLAGS.batch_size, set_size - i)
    batch_filenames = file_names[i:i+batch_size]
    predict_fingerprints = audio_processor.get_predicion_data(
        batch_filenames, model_settings, sess)

    test_accuracy = sess.run(
        predicted_indices,
        feed_dict={
            fingerprint_input: predict_fingerprints,
            #ground_truth_input: test_ground_truth,
            dropout_prob: 1.0
        })
    
    '''
    if batch_num % 100 == 0:
        print ("i {}, batch num {}, batch size = {}".format(i, batch_num, batch_size))
        print (batch_filenames[10:20])
        print (test_accuracy[10:20])
    '''
    for j in range(batch_size):
        subm[i+j] = (subm[i+j][0],words_list[test_accuracy[j]])
    batch_num += 1



/notebooks/data/test/audio


In [15]:
for i in range(10,20):
    print (batch_filenames[i])
    print (words_list[test_accuracy[i]])
    print (subm[i])

clip_06786ab06.wav
yes
('clip_3c0b101e0.wav', 'right')
clip_a629a0a8c.wav
_unknown_
('clip_24d52f235.wav', 'off')
clip_58b5acafb.wav
go
('clip_47cc3ea4e.wav', '_silence_')
clip_c0e7e1556.wav
_unknown_
('clip_28ce90f4b.wav', '_unknown_')
clip_62e2c644e.wav
_unknown_
('clip_d83092849.wav', '_silence_')
clip_0768f2c6f.wav
_unknown_
('clip_798b8971d.wav', '_unknown_')
clip_1a9ad5f94.wav
up
('clip_9c432a821.wav', 'yes')
clip_592bb8eab.wav
_unknown_
('clip_c2e9fd918.wav', '_unknown_')
clip_bb779fa23.wav
stop
('clip_67a34437e.wav', '_silence_')
clip_866c6ffb8.wav
_silence_
('clip_5ef10de58.wav', 'on')


In [16]:
test_accuracy

array([ 2,  0,  0,  9,  1,  0,  1, 10,  5,  2,  2,  1, 11,  1,  1,  1,  4,
        1, 10,  0,  0,  6,  0,  1,  1,  6,  8,  1,  1,  3,  0,  9,  2,  6,
        0,  4,  6,  1])

In [17]:
print (subm[:10])

[('clip_55e803b0c.wav', '_unknown_'), ('clip_f248fa402.wav', '_silence_'), ('clip_04476ba6d.wav', '_unknown_'), ('clip_aa853b568.wav', 'go'), ('clip_b8eb354f8.wav', '_unknown_'), ('clip_541afdf9b.wav', '_silence_'), ('clip_04fcbf5e4.wav', 'on'), ('clip_3ec4ed90b.wav', 'yes'), ('clip_4bc631523.wav', '_unknown_'), ('clip_41a2b5699.wav', '_silence_')]


### Create submission file

In [18]:
# submission1.csv : sample submission
# submission2.csv : TensorFlow example code prediction, trained with Speech Commands Data Set v0.01
# submission3.csv : TensorFlow example code prediction, trained with Kaggle Data Set
submission_file_name = DATA_DIR+'/submission3.csv'
print (submission_file_name)
#np.savetxt(submission_file_name, subm, header='fname,label', comments='')
with open(submission_file_name, 'wb') as outfile:
    wr = csv.writer(outfile)
    wr.writerow(('fname', 'label'))
    for i in range(len(subm)):
        wr.writerow(subm[i])

/notebooks/data/submission3.csv


In [19]:
from IPython.display import FileLink
FileLink(submission_file_name)

/notebooks/data/submission3.csv